# Praca domowa 2
## Simple convolution

In [1]:
import cv2 
from PIL import Image
import numpy as np

In [2]:
def processImage(image, color = False): 
    image = cv2.imread(image) 
    if color:
        image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    else:
        image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY)  
    return image

image = processImage('Image.jpg')

In [3]:
def convolve2D(image, kernels, strides=1):

    x_ker, y_ker = kernels[0].shape
    
    # Jeśli obrazek jest kolorowy to wtedy jest troche inaczej 
    if len(image.shape) == 3:
        x_size, y_size, z_size = image.shape
    else: 
        x_size, y_size = image.shape
        z_size = 1
    
    # Wymiary macierzy wyjściowej
    x_out = int(((x_size - x_ker) / strides))+ 1
    y_out = int(((y_size - y_ker) / strides)) + 1
    z_out = z_size
    
    # Lista macierzy wyjściowych. Tyle wyników ile różnych filtrów
    output_list = [0] * len(kernels)
    out_index = 0
    
    for kernel in kernels:
        output_RGB = np.zeros((x_out, y_out, z_out))
        for i in range(z_out):
            for y in range(0, y_size - y_ker, strides):
                for x in range(0, x_size - x_ker, strides):
                    # Warunek if aby mogła być jedna funkcja do kolorowych i zwykłych obrazków
                    if len(image.shape) == 3:
                        output_RGB[x// strides, y// strides, 2 - i] = int((kernel * image[x: x + x_ker, y: y + y_ker, i]).sum())
                    else: 
                        output_RGB[x// strides, y// strides] = int((kernel * image[x: x + x_ker, y: y + y_ker]).sum())
                    
        output_list[out_index] = output_RGB
        out_index += 1

    return output_list

### Obrazek czarno-biały 

In [4]:
kernel = [np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])]
output = convolve2D(image, kernel)
#cv2.imwrite('pies1.jpeg', output[0])


<img src="pies1.jpeg">

### Obrazek czarno-biały i wiele filtów 

In [5]:
kernel = [np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]]), np.array([[1, -1, 0], [-2, 8, 0], [0, -2, -3]])]
output = convolve2D(image, kernel)
#cv2.imwrite('pies4.jpeg', output[1])


<img src="pies4.jpeg">

### Obrazek kolorowy

In [6]:
kernel = [np.array([[1, -1, 0], [-2, 8, 0], [0, -2, -3]])]
image = cv2.imread('Image.jpg') 
image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB) 
output = convolve2D(image, kernel)
#cv2.imwrite('pies2.jpeg', output[0])


<img src="pies2.jpeg">

## Convolution using matrix multiplication

In [7]:
def im2col(x,hh,ww,stride):
    # x - macierz wejściowa
    # hh - wysokość kernela
    # ww - szerokość kernela
    
    h,w = x.shape
    new_h = (h-hh) // stride + 1
    new_w = (w-ww) // stride + 1
    
    # Tworzenie macierzy przekształconej gdzie każde okienko jest jednym wierszem
    col = np.zeros([new_h*new_w,hh*ww])

    for i in range(new_h):
       for j in range(new_w):
           patch = x[...,i*stride:i*stride+hh,j*stride:j*stride+ww]
           col[i*new_w+j,:] = np.reshape(patch,-1)
    return col

In [10]:
def convolve2D_matrix(image,kernels,stride = 1):
    
    output_list = [0] * len(kernels)
    out_index = 0
    
    x_ker, y_ker = kernels[0].shape
    output_shape_y = (image.shape[1] - y_ker) + 1
    output_shape_x = (image.shape[0] - x_ker) + 1
    
    for kernel in kernels:
        mod_matrix = im2col(image, x_ker, y_ker, stride)
        
        # Mnożymy macierz przekształconą z spłaszczonym wektorem filtra
        im2col_conv = np.dot(mod_matrix, kernel.flatten())
        
        # Przekształcamy wektor wynikowy do postaci macierzy 
        im2col_conv = im2col_conv.reshape(output_shape_x,output_shape_y)
        
        output_list[out_index] = im2col_conv
        out_index += 1
    return output_list
        

In [11]:
kernel = [np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])]
image = cv2.imread('Image.jpg') 
image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) 
output = convolve2D_matrix(image, kernel, 1)
#cv2.imwrite('pies3.jpeg', output[0])

<img src="pies3.jpeg">

## Porównanie czasów 

In [12]:
from timeit import default_timer as timer
kernel = [np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])]

start = timer()
output = convolve2D(image, kernel)
end = timer()
print(" Czas zwykłej metody: ", end - start)

start = timer()
output = convolve2D_matrix(image, kernel)
end = timer()
print(" Czas metody z mnożeniem macierzy: ", end - start)

 Czas zwykłej metody  2.4477066240000056
 Czas zwykłej metody  1.2656276669999897


Mnożenie macierzowe przyśpiesza operacje konwolucji, mimo wszystko musimy przejść po macierzy obrazka, aby utowrzyć macierz zmodyfikowaną, jest to czasochłonna operacja, ale samo mnożenie odbywa się szybciej.